<a href="https://colab.research.google.com/github/taka-too/family_num_spiral/blob/main/family_num_spiral.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 スパイラル回覧PDF作成ツール（家庭番号自動配置）

このツールは、父母会・保護者会等で配布される「回覧」冊子の表紙として使える**スパイラル配置PDF**を自動生成するGoogle Colabノートブックです。

---

## 🧩 機能一覧

- ✔️ **家庭番号をスパイラル状に自動配置**
- ✔️ **100〜500番台ごとにページを分割**
- ✔️ **各ページに日本語タイトル・説明文を中央配置**
- ✔️ **家庭番号は円で囲み、ゼロ埋めで表示（例: 101 → 101）**
- ✔️ **PDFは `/tmp/` に生成され、完了後に自動ダウンロード**
- ✔️ **スパイラル間隔やフォントサイズをGUIで調整可能**

使用手順はページ下部にあります。

In [ ]:
# ✅ 初回のみ：必要なライブラリ・フォントをインストール
!apt-get -y install fonts-noto-cjk > /dev/null
!pip install ipywidgets openpyxl > /dev/null

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.integrate import quad
from scipy.optimize import root_scalar
import matplotlib.font_manager as fm
import matplotlib as mpl
from google.colab import files
import tempfile, os, io
import ipywidgets as widgets
from IPython.display import display

# 🎌 フォント設定（PDF埋め込み）
mpl.rcParams["pdf.fonttype"] = 42
mpl.rcParams["ps.fonttype"] = 42
jp_font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
jp_font = fm.FontProperties(fname=jp_font_path)

# 🎛️ GUIウィジェット
title_input = widgets.Text(value="回覧", description="中央タイトル:")
font_size_input = widgets.IntSlider(value=10, min=6, max=24, step=1, description="説明文サイズ:")
circle_size_input = widgets.FloatSlider(value=1.5, min=0.5, max=3.0, step=0.1, description="円の大きさ:")
spacing_input = widgets.FloatSlider(value=3.0, min=1.0, max=10.0, step=0.5, description="スパイラル間隔:")
output_filename_input = widgets.Text(value="spiral_kairan.pdf", description="出力ファイル名:")
upload_button = widgets.FileUpload(accept=".xlsx", multiple=False, description="Excelアップロード")
run_button = widgets.Button(description="PDF作成 ▶")
uploaded_file_label = widgets.Text(value="", description="選択ファイル名", disabled=True)

# 👀 表示
display(title_input, font_size_input, circle_size_input, spacing_input, output_filename_input, upload_button, uploaded_file_label, run_button)

# 📥 ファイル名表示
def on_file_upload(change):
    if upload_button.value:
        fname = list(upload_button.value.keys())[0]
        uploaded_file_label.value = fname
upload_button.observe(on_file_upload, names="value")

# 🌀 スパイラル描画 → PDF出力（番台ごと）
def generate_spiral_pdf_grouped(file_stream, filename, title, font_size, circle_radius, spacing):
    # スパイラル関数定義
    a, n_power, r0, theta0 = 2.0, 0.8, 10, np.pi / 2
    def r(theta): return a * (theta**n_power) + r0
    def dr(theta): return a * n_power * theta**(n_power - 1)
    def arc_integrand(theta): return np.sqrt(r(theta)**2 + dr(theta)**2)
    def arc_length(theta): return quad(arc_integrand, theta0, theta)[0]
    def solve_theta(L): return theta0 if L == 0 else root_scalar(lambda t: arc_length(t) - L, bracket=[theta0 + 1e-6, theta0 + 1000], method='brentq').root

    # Excel読み込み（1行目もデータ扱い）
    df = pd.read_excel(file_stream, header=None, usecols=[0], names=["家庭番号"])
    df["家庭番号"] = df["家庭番号"].astype(int)

    # 出力先ファイル
    temp_pdf_path = os.path.join(tempfile.gettempdir(), filename)
    pdf = PdfPages(temp_pdf_path)

    # 番台ごとに分けて描画
    for base in range(100, 600, 100):
        numbers = df[(df["家庭番号"] >= base) & (df["家庭番号"] < base + 100)]["家庭番号"].tolist()
        if not numbers:
            continue

        theta_vals = [solve_theta(i * spacing) for i in range(len(numbers))]
        r_vals = [r(t) for t in theta_vals]
        x_vals = [r_val * np.cos(t) for r_val, t in zip(r_vals, theta_vals)]
        y_vals = [r_val * np.sin(t) for r_val, t in zip(r_vals, theta_vals)]

        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_aspect('equal')
        ax.axis('off')

        for i, num in enumerate(numbers):
            circle = plt.Circle((x_vals[i], y_vals[i]), circle_radius, edgecolor='black', facecolor='white')
            ax.add_patch(circle)
            ax.text(x_vals[i], y_vals[i], f"{num:03d}", ha='center', va='center', fontsize=8, fontproperties=jp_font)

        # テキスト
        ax.text(0, -2, title, fontsize=40, ha='center', va='center', weight='bold', fontproperties=jp_font)
        ax.text(0, -7, "読み終わりましたら\nチェックしてお戻しください", fontsize=font_size, ha='center', va='top', fontproperties=jp_font)
        ax.text(0, 7, "おうちのひとにみせてね！\nみんなでリレーしよう！\nどこまでいけるかな？", fontsize=font_size, ha='center', va='top', fontproperties=jp_font)

        ax.set_xlim(min(x_vals)-2, max(x_vals)+2)
        ax.set_ylim(min(y_vals)-2, max(y_vals)+2)

        pdf.savefig(fig)
        plt.close(fig)
        print(f"✅ {base}番台ページを追加しました")

    # 保存とダウンロード
    pdf.close()
    files.download(temp_pdf_path)

# ▶️ 実行ボタン処理
def on_run_button_clicked(b):
    if upload_button.value:
        uploaded_filename = list(upload_button.value.keys())[0]
        file_stream = io.BytesIO(upload_button.value[uploaded_filename]["content"])
        generate_spiral_pdf_grouped(
            file_stream,
            output_filename_input.value,
            title_input.value,
            font_size_input.value,
            circle_size_input.value,
            spacing_input.value
        )
run_button.on_click(on_run_button_clicked)


---

## 📥 使用手順

### セルを実行すると、必要なライブラリとフォントが自動インストールされます

GUIが表示されるまで少し時間がかかります。

### 各種パラメータをGUIで入力

- **中央タイトル**：スパイラルの中心に表示されるタイトル（例：「回覧」）
- **説明文サイズ**：上下の補足文のフォントサイズ
- **円の大きさ**：番号を囲む円の直径
- **スパイラル間隔**：渦巻きの巻きの強さ
- **出力ファイル名**：生成されるPDFファイル名（例：`spiral_kairan.pdf`）

### Excelアップロード」で `.xlsx` ファイルを選択

- **1列目に家庭番号を記載したExcelファイル**をご用意ください。
- 見出し行は不要です。全行が番号として扱われます。

### 「PDF作成 ▶」ボタンをクリック

- 番台（100〜500番台）ごとにページを分けてPDFを生成します。
- 完了後、自動でファイルをダウンロードします。

---

## 📂 入力Excelファイルの例

https://github.com/taka-too/family_num_spiral/raw/refs/heads/main/family_number.xlsx

---

## 📤 出力PDFの特徴

出力ファイルの例
https://github.com/taka-too/family_num_spiral/raw/refs/heads/main/spiral_kairan-2.pdf

- ページごとに：
  - スパイラル配置された家庭番号（同じ番台内）
  - 中央に「回覧」などのタイトル
  - 上下に説明文

- フォント：Google Colabに内蔵されている Noto Sans CJK JP
- PDF内に**フォントも埋め込み済み**なので文字化けの心配なし


---

## ✅ 注意点

- Excelは `.xlsx` 形式のみ対応
- 家庭番号はすべて整数で記載してください
- 番台に該当する番号がないページは出力されません
- PDFは `/tmp/` に一時保存されるため、**セッションが終了すると破棄されます**